# Ingeniería de datos


In [7]:
# Librerias a usar
import os
import time
import pandas as pd
import numpy as np

In [8]:
df1 = "//192.168.1.2/cenyt-proyectos/CEN-221_ANDA/2.INFOENTRADA/5. Inspeccion Redes/2025-09-15/ANDA-REDES/registros_exportados_1758071892733.xlsx"

In [9]:
def limpiar_rutas_imagenes(ruta_archivo_excel, columnas_a_limpiar, prefijo_a_eliminar):
    """
    Lee un archivo de Excel, limpia las rutas de imágenes en las columnas especificadas
    y guarda el resultado en un nuevo archivo.

    Args:
        ruta_archivo_excel (str): La ruta completa del archivo de Excel de origen.
        columnas_a_limpiar (list): Una lista de los nombres de las columnas que contienen las rutas.
        prefijo_a_eliminar (str): El texto a eliminar de las rutas de las imágenes.

    Returns:
        str: La ruta del nuevo archivo de Excel creado.
    """
    
    # 1. Leer el archivo de Excel
    try:
        df = pd.read_excel(ruta_archivo_excel)
        print("Archivo de Excel leído exitosamente.")
    except FileNotFoundError:
        print(f"Error: El archivo no se encuentra en la ruta especificada: {ruta_archivo_excel}")
        return None
    
    # 2. Limpiar las columnas especificadas
    for columna in columnas_a_limpiar:
        if columna in df.columns:
            # Reemplazar el texto no deseado y asegurarse de que el tipo de dato sea string
            df[columna] = df[columna].astype(str).apply(lambda x: x.replace(prefijo_a_eliminar, '').strip())
            print(f"Columna '{columna}' procesada.")
        else:
            print(f"Advertencia: La columna '{columna}' no se encontró en el archivo.")
            
    # 3. Construir la nueva ruta del archivo de salida
    nombre_archivo, extension = os.path.splitext(ruta_archivo_excel)
    nueva_ruta_archivo = f"{nombre_archivo}_limpio{extension}"
    
    # 4. Guardar los cambios en un nuevo archivo de Excel
    df.to_excel(nueva_ruta_archivo, index=False)
    print(f"Datos limpios guardados en: {nueva_ruta_archivo}")
    
    return nueva_ruta_archivo

# --- Configuración y ejecución ---

# Define la ruta de tu archivo de Excel y el prefijo a eliminar
ruta_excel = df1# 👈 ¡Cambia esta ruta por la de tu archivo!
prefijo_a_quitar = "/storage/emulated/0/Download/RegistrosExportados/"

# Define las columnas a procesar
columnas_de_fotos = ["Foto1", "Foto2", "Foto3", "Foto4", "Foto5", "Foto6", "Foto7", "Foto8", "Foto9", "Foto10", "Foto11", "Foto12", "Foto13", "Foto14", "Foto15", "Foto16"]

# Llama a la función para limpiar las rutas
limpiar_rutas_imagenes(ruta_excel, columnas_de_fotos, prefijo_a_quitar)

Archivo de Excel leído exitosamente.
Columna 'Foto1' procesada.
Columna 'Foto2' procesada.
Columna 'Foto3' procesada.
Columna 'Foto4' procesada.
Columna 'Foto5' procesada.
Columna 'Foto6' procesada.
Columna 'Foto7' procesada.
Columna 'Foto8' procesada.
Columna 'Foto9' procesada.
Columna 'Foto10' procesada.
Columna 'Foto11' procesada.
Columna 'Foto12' procesada.
Columna 'Foto13' procesada.
Columna 'Foto14' procesada.
Columna 'Foto15' procesada.
Columna 'Foto16' procesada.
Datos limpios guardados en: //192.168.1.2/cenyt-proyectos/CEN-221_ANDA/2.INFOENTRADA/5. Inspeccion Redes/2025-09-15/ANDA-REDES/registros_exportados_1758071892733_limpio.xlsx


'//192.168.1.2/cenyt-proyectos/CEN-221_ANDA/2.INFOENTRADA/5. Inspeccion Redes/2025-09-15/ANDA-REDES/registros_exportados_1758071892733_limpio.xlsx'

In [7]:
import os
import pandas as pd
import glob

# --- CONFIGURACIÓN DE RUTAS ---
# La ruta de red de los datos de entrada
ruta_carpeta_entrada = r'C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\DATOS CRUDOS'

# La nueva ruta donde se guardará el archivo de salida
ruta_salida = r'C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\DATOS FINALES' # <--- CAMBIA ESTO A TU RUTA DESEADA
nombre_archivo_salida = 'proyecto_consolidado.xlsx'
ruta_completa_salida = os.path.join(ruta_salida, nombre_archivo_salida)

# --- CONFIGURACIÓN DE COLUMNAS Y DATOS ---
columnas_requeridas = ['ID_GPS1', 'ID_GPS2', 'ID_GPS3']
ruta_base_a_eliminar = '/storage/emulated/0/Download/RegistrosExportados/'
lista_dataframes = []

# --- 1. PROCESAR Y CONSOLIDAR ARCHIVOS ---
try:
    archivos = glob.glob(os.path.join(ruta_carpeta_entrada, '*.csv')) + \
               glob.glob(os.path.join(ruta_carpeta_entrada, '*.xlsx'))

    if not archivos:
        print(f"No se encontraron archivos .csv o .xlsx en la ruta: {ruta_carpeta_entrada}")
    else:
        for archivo in archivos:
            nombre_archivo = os.path.basename(archivo)
            print(f"Procesando archivo: {nombre_archivo}")

            try:
                if archivo.endswith('.csv'):
                    try:
                        df = pd.read_csv(archivo, encoding='utf-8', sep=';', on_bad_lines='skip')
                    except pd.errors.ParserError:
                        df = pd.read_csv(archivo, encoding='latin1', sep=';', on_bad_lines='skip')
                elif archivo.endswith('.xlsx'):
                    df = pd.read_excel(archivo)
                else:
                    print(f"  -> Tipo de archivo no soportado: {nombre_archivo}. Omitiendo.")
                    continue
                
                for col in columnas_requeridas:
                    if col not in df.columns:
                        print(f"  -> La columna '{col}' no existe. Creando y llenando con valores nulos.")
                        df[col] = pd.NA

                lista_dataframes.append(df)
            except Exception as e:
                print(f"  -> Error al leer o procesar el archivo {nombre_archivo}: {e}")

    if lista_dataframes:
        dataframe_final = pd.concat(lista_dataframes, ignore_index=True)
        print("\n¡Concatenación exitosa! DataFrame unificado creado.")
        
        # --- 2. PROCESAR LA COLUMNA 'Otras_Observaciones' Y LIMPIAR LA RUTA BASE ---
        if 'Otras_Observaciones' in dataframe_final.columns:
            print("\nProcesando la columna 'Otras_Observaciones'...")
            dataframe_final['Otras_Observaciones'] = dataframe_final['Otras_Observaciones'].fillna('')
            fotos_df = dataframe_final['Otras_Observaciones'].str.split(',', expand=True)
            
            # Limpiar la ruta base de las nuevas columnas de fotos
            for col in fotos_df.columns:
                fotos_df[col] = fotos_df[col].str.replace(ruta_base_a_eliminar, '', regex=False)
            
            fotos_df.columns = [f'Foto{i+1}' for i in fotos_df.columns]
            
            dataframe_final = pd.concat([dataframe_final, fotos_df], axis=1)
            dataframe_final.drop('Otras_Observaciones', axis=1, inplace=True)
            print("Columnas de fotos creadas y limpiadas exitosamente.")
        else:
            print("\nLa columna 'Otras_Observaciones' no se encontró. Este paso será omitido.")
            
        print("\nDataFrame final (primeras 5 filas):")
        print(dataframe_final.head())

        # --- 3. EXPORTAR EL DATAFRAME A UN ARCHIVO DE EXCEL ---
        if not os.path.exists(ruta_salida):
            os.makedirs(ruta_salida)
        
        dataframe_final.to_excel(ruta_completa_salida, index=False)
        print(f"\n¡Proyecto finalizado! El archivo se ha guardado exitosamente en: {ruta_completa_salida}")

    else:
        print("\nNo se pudo unificar ningún archivo. El DataFrame final está vacío.")
        dataframe_final = pd.DataFrame()

except Exception as e:
    print(f"Ocurrió un error al acceder a la ruta de red o guardar el archivo: {e}")
    dataframe_final = pd.DataFrame()

Procesando archivo: registros_exportados_1757459149029.csv
Procesando archivo: registros_exportados_1758170527743.csv
Procesando archivo: New Microsoft Excel Worksheet.xlsx

¡Concatenación exitosa! DataFrame unificado creado.

Procesando la columna 'Otras_Observaciones'...
Columnas de fotos creadas y limpiadas exitosamente.

DataFrame final (primeras 5 filas):
     ID            FechaHora NombreResponsable      Area           Circuito  \
0  39.0  2025-09-09 10:32:12             ANDA   EB3-EB2   SE CEL-Anda - EB2   
1  38.0  2025-09-09 10:24:27             ANDA   EB3-EB2   SE CEL-Anda - EB2   
2  37.0  2025-09-09 10:09:15             ANDA   EB3-EB2   SE CEL-Anda - EB2   
3  36.0  2025-09-09 10:02:35             ANDA   EB3-EB2   SE CEL-Anda - EB2   
4  35.0  2025-09-09 09:55:49             ANDA   EB3-EB2   SE CEL-Anda - EB2   

  Circuito_Observaciones Estructura_Tag  Latitud_GPS  Longitud_GPS  \
0                    NaN            114    13.838282    -89.262744   
1                    N

In [8]:
import pandas as pd
import openpyxl
import os
from openpyxl.drawing.image import Image

# --- CONFIGURACIÓN DE RUTAS ---
# La ruta del archivo de Excel consolidado que creamos
ruta_archivo_entrada = r'C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\DATOS FINALES\proyecto_consolidado.xlsx' # <--- ASEGÚRATE DE QUE SEA LA RUTA CORRECTA

# La ruta donde se encuentran las imágenes
ruta_carpeta_fotos = r'C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_3'

# El nombre del nuevo archivo de Excel con las imágenes
nombre_archivo_salida_con_fotos = 'proyecto_consolidado_con_fotos.xlsx'

# Asegúrate de que la ruta de salida sea la misma que la de entrada, o una nueva si lo prefieres
ruta_completa_salida = os.path.join(os.path.dirname(ruta_archivo_entrada), nombre_archivo_salida_con_fotos)

# --- 1. LEER EL ARCHIVO CONSOLIDADO ---
print("Leyendo el archivo de Excel consolidado...")
try:
    df = pd.read_excel(ruta_archivo_entrada)
except FileNotFoundError:
    print(f"Error: El archivo de entrada no se encontró en la ruta: {ruta_archivo_entrada}")
    exit()

# --- 2. PREPARAR EL ARCHIVO DE EXCEL CON OPENPYXL ---
# Creamos un objeto de Excel a partir del DataFrame
workbook = openpyxl.Workbook()
sheet = workbook.active

# Escribimos los encabezados del DataFrame en la hoja de Excel
for col_num, column_title in enumerate(df.columns, 1):
    sheet.cell(row=1, column=col_num, value=column_title)

# Escribimos los datos del DataFrame en la hoja de Excel
for row_num, row_data in enumerate(df.values, 2):
    for col_num, cell_data in enumerate(row_data, 1):
        sheet.cell(row=row_num, column=col_num, value=cell_data)

print("Estructura de Excel preparada.")

# --- 3. INCRUSTAR LAS IMÁGENES ---
print("Buscando e incrustando imágenes...")
# Identificamos las columnas que contienen las rutas de las fotos (ej. 'Foto1', 'Foto2')
columnas_fotos = [col for col in df.columns if col.startswith('Foto')]

if not columnas_fotos:
    print("No se encontraron columnas de fotos ('Foto1', 'Foto2', etc.) en el archivo.")
else:
    for row_idx, row in df.iterrows():
        excel_row = row_idx + 2  # Las filas de Excel comienzan en 1 y los datos en la fila 2
        for foto_col in columnas_fotos:
            # Obtener el índice de la columna en el DataFrame
            df_col_idx = df.columns.get_loc(foto_col)
            excel_col = df_col_idx + 1 # Las columnas de Excel comienzan en 1
            
            # Obtener el nombre de la foto de la celda
            nombre_foto = row[foto_col]
            
            if pd.notna(nombre_foto) and nombre_foto: # Verificar si la celda no está vacía
                ruta_completa_foto = os.path.join(ruta_carpeta_fotos, nombre_foto)
                
                if os.path.exists(ruta_completa_foto):
                    try:
                        # Incrustar la imagen en la celda
                        img = Image(ruta_completa_foto)
                        # Opcional: ajustar el tamaño de la imagen si es necesario
                        # img.width = 100
                        # img.height = 100
                        sheet.add_image(img, sheet.cell(row=excel_row, column=excel_col).coordinate)
                        print(f"  -> Imagen incrustada: {nombre_foto} en la celda {sheet.cell(row=excel_row, column=excel_col).coordinate}")
                    except Exception as e:
                        print(f"  -> Error al incrustar la imagen {nombre_foto}: {e}")
                else:
                    print(f"  -> Archivo no encontrado: {ruta_completa_foto}")

print("\nProceso de incrustación de imágenes finalizado.")

# --- 4. GUARDAR EL NUEVO ARCHIVO DE EXCEL ---
try:
    workbook.save(ruta_completa_salida)
    print(f"\n¡Éxito! El archivo con las imágenes se ha guardado en: {ruta_completa_salida}")
except Exception as e:
    print(f"Error al guardar el archivo: {e}")

Leyendo el archivo de Excel consolidado...
Estructura de Excel preparada.
Buscando e incrustando imágenes...
  -> Archivo no encontrado: C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_3\JPEG_20250909_103657_8675553972885569530.jpg
  -> Archivo no encontrado: C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_3\JPEG_20250909_103709_1748070493310971728.jpg
  -> Archivo no encontrado: C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_3\JPEG_20250909_103726_3082911714446878058.jpg
  -> Archivo no encontrado: C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_3\JPEG_20250909_103753_1288292783344713862.jpg
  -> Archivo no encontrado: C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_3\JPEG_20250909_103843_166171686854279677.jpg
  -> Archivo no encontrado: C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_3\JPEG_20250909_102959_4309296348802395234.jpg
  -> Archivo no encontrado: C:\Use

In [9]:
import pandas as pd
import openpyxl
import os
from openpyxl.drawing.image import Image

# --- CONFIGURACIÓN ---
ruta_archivo_entrada = r'C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\DATOS FINALES\proyecto_consolidado.xlsx'
ruta_carpeta_fotos = r'C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_3'
nombre_archivo_salida_con_fotos = 'proyecto_consolidado_con_fotos.xlsx'
ruta_completa_salida = os.path.join(os.path.dirname(ruta_archivo_entrada), nombre_archivo_salida_con_fotos)

# --- 1. LEER EL ARCHIVO CONSOLIDADO ---
print("Leyendo el archivo de Excel consolidado...")
df = pd.read_excel(ruta_archivo_entrada)

# Columnas de fotos
columnas_fotos = [col for col in df.columns if col.startswith('Foto')]
if not columnas_fotos:
    print("⚠️ No se encontraron columnas que empiecen con 'Foto'.")
    columnas_fotos = []

# --- 2. CREAR WORKBOOK ---
workbook = openpyxl.Workbook()
# Borro la hoja por defecto
workbook.remove(workbook.active)

# --- 3. CREAR UNA HOJA POR CADA CIRCUITO ---
if "Circuito" not in df.columns:
    print("⚠️ No existe la columna 'Circuito'. Se usará una sola hoja.")
    circuitos = {"General": df}
else:
    circuitos = {c: df[df["Circuito"] == c] for c in df["Circuito"].unique()}

for circuito, df_circ in circuitos.items():
    print(f"Creando hoja: {circuito}")
    sheet = workbook.create_sheet(title=str(circuito))

    # Encabezados
    for col_num, column_title in enumerate(df_circ.columns, 1):
        sheet.cell(row=1, column=col_num, value=column_title)

    # Datos + imágenes
    for row_idx, row in df_circ.iterrows():
        excel_row = list(df_circ.index).index(row_idx) + 2  # fila relativa dentro de esta hoja
        for col_num, cell_data in enumerate(row, 1):
            sheet.cell(row=excel_row, column=col_num, value=cell_data)

        # Procesar imágenes
        for foto_col in columnas_fotos:
            df_col_idx = df_circ.columns.get_loc(foto_col)
            excel_col = df_col_idx + 1
            nombre_foto = row[foto_col]

            if pd.notna(nombre_foto) and nombre_foto:
                ruta_foto = os.path.join(ruta_carpeta_fotos, nombre_foto)
                if os.path.exists(ruta_foto):
                    try:
                        img = Image(ruta_foto)
                        # Redimensionar a 100x100
                        img.width, img.height = 100, 100
                        coord = sheet.cell(row=excel_row, column=excel_col).coordinate
                        sheet.add_image(img, coord)
                    except Exception as e:
                        print(f"Error al incrustar {nombre_foto}: {e}")

    # --- 4. AJUSTAR ANCHO Y ALTO ---
    # Ajustar filas
    for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
        sheet.row_dimensions[row[0].row].height = 75  # ~100px

    # Ajustar columnas
    for col in range(1, sheet.max_column + 1):
        sheet.column_dimensions[openpyxl.utils.get_column_letter(col)].width = 15  # ~100px

# --- 5. GUARDAR ---
try:
    workbook.save(ruta_completa_salida)
    print(f"\n✅ Archivo final guardado en: {ruta_completa_salida}")
except Exception as e:
    print(f"Error al guardar: {e}")


Leyendo el archivo de Excel consolidado...
Creando hoja: SE CEL-Anda - EB2
Creando hoja: EB1 - SE Santa Ana
Creando hoja: na

✅ Archivo final guardado en: C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\DATOS FINALES\proyecto_consolidado_con_fotos.xlsx


In [3]:
import os
import shutil
from PIL import Image

# Ruta de la carpeta donde están las imágenes
carpeta = r"\\192.168.1.2\cenyt-proyectos\CEN-221_ANDA\2.INFOENTRADA\5. Inspeccion Redes\2025-09-15\ANDA-REDES\FOTOS"

# Carpeta de salida
carpeta_salida = r"\\192.168.1.2\cenyt-proyectos\CEN-221_ANDA\2.INFOENTRADA\5. Inspeccion Redes\2025-09-15\ANDA-REDES\Comprimidas"
os.makedirs(carpeta_salida, exist_ok=True)

# Tamaño máximo permitido en KB
limite_kb = 499

for archivo in os.listdir(carpeta):
    ruta = os.path.join(carpeta, archivo)

    # Verificar si es archivo y una imagen
    if os.path.isfile(ruta) and archivo.lower().endswith(('.jpg', '.jpeg', '.png')):
        tamano_kb = os.path.getsize(ruta) / 1024
        print(f"{archivo} -> {tamano_kb:.2f} KB")

        destino = os.path.join(carpeta_salida, archivo)

        if tamano_kb <= limite_kb:
            # Copiar directamente si no excede el límite
            shutil.copy2(ruta, destino)
            print(f"   → Copiada sin cambios ({tamano_kb:.2f} KB)")
        else:
            # Comprimir si excede el límite
            try:
                img = Image.open(ruta)

                if archivo.lower().endswith(('.jpg', '.jpeg')):
                    img.save(destino, optimize=True, quality=70)
                else:  # PNG → optimizar
                    img.save(destino, optimize=True)

                nuevo_tamano = os.path.getsize(destino) / 1024
                print(f"   → Comprimida: {nuevo_tamano:.2f} KB")

            except Exception as e:
                print(f"Error con {archivo}: {e}")



JPEG_20250913_111235_3250883094701253140.jpg -> 5855.35 KB
   → Comprimida: 2294.34 KB
JPEG_20250913_111221_5748109541630654790.jpg -> 3548.85 KB
   → Comprimida: 1197.78 KB
JPEG_20250913_111129_1359705467680473041.jpg -> 5964.25 KB
   → Comprimida: 2363.53 KB
JPEG_20250913_110338_5898135733816280796.jpg -> 4363.29 KB
   → Comprimida: 1608.91 KB
JPEG_20250913_110317_5329668562513601603.jpg -> 4577.55 KB
   → Comprimida: 1708.54 KB
JPEG_20250913_110254_960942644729947289.jpg -> 4716.71 KB
   → Comprimida: 1775.04 KB
JPEG_20250913_105537_2639509893398097532.jpg -> 4225.35 KB
   → Comprimida: 1504.62 KB
JPEG_20250913_105505_1820768442477964715.jpg -> 5417.43 KB
   → Comprimida: 2113.77 KB
JPEG_20250913_105433_6535652925998488234.jpg -> 5448.69 KB
   → Comprimida: 2081.37 KB
JPEG_20250913_104712_558431206323329141.jpg -> 3291.01 KB
   → Comprimida: 1045.44 KB
JPEG_20250913_104646_197311876332633740.jpg -> 3341.24 KB
   → Comprimida: 1048.67 KB
JPEG_20250913_104632_6748894105590554641.jpg -

In [3]:
import os
import shutil
from PIL import Image

# Carpetas
carpeta = r"C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas"
carpeta_salida = r"C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_2"
os.makedirs(carpeta_salida, exist_ok=True)

# Límite en KB
limite_kb = 49

# Listado de archivos originales y ya procesados
archivos_entrada = {f for f in os.listdir(carpeta) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}
archivos_salida = {f for f in os.listdir(carpeta_salida) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}

# Calcular los pendientes
pendientes = archivos_entrada - archivos_salida

print(f"Total imágenes en carpeta: {len(archivos_entrada)}")
print(f"Ya procesadas: {len(archivos_salida)}")
print(f"Pendientes: {len(pendientes)}")

for archivo in pendientes:
    ruta = os.path.join(carpeta, archivo)
    destino = os.path.join(carpeta_salida, archivo)

    if os.path.isfile(ruta):
        tamano_kb = os.path.getsize(ruta) / 1024
        print(f"{archivo} -> {tamano_kb:.2f} KB")

        if tamano_kb <= limite_kb:
            # Copiar directamente
            shutil.copy2(ruta, destino)
            print(f"   → Copiada sin cambios ({tamano_kb:.2f} KB)")
        else:
            # Comprimir
            try:
                img = Image.open(ruta)
                if archivo.lower().endswith(('.jpg', '.jpeg')):
                    img.save(destino, optimize=True, quality=70)
                else:
                    img.save(destino, optimize=True)

                nuevo_tamano = os.path.getsize(destino) / 1024
                print(f"   → Comprimida: {nuevo_tamano:.2f} KB")

            except Exception as e:
                print(f"Error con {archivo}: {e}")


Total imágenes en carpeta: 523
Ya procesadas: 0
Pendientes: 523
JPEG_20250916_100034_4279327063874974096.jpg -> 1364.22 KB
   → Comprimida: 1364.22 KB
IMG_20250917_092541_6.jpg -> 2122.57 KB
   → Comprimida: 2121.91 KB
IMG_20250917_114707_0.jpg -> 1687.73 KB
   → Comprimida: 1686.37 KB
JPEG_20250916_152333_8056945582870247404.jpg -> 1579.24 KB
   → Comprimida: 1579.25 KB
JPEG_20250916_090227_3115831209829880089.jpg -> 1618.76 KB
   → Comprimida: 1618.39 KB
IMG_20250916_112256_11.jpg -> 214.56 KB
   → Comprimida: 214.44 KB
IMG_20250915_181638_1.jpg -> 459.23 KB
   → Comprimida: 178.97 KB
IMG_20250913_155434_2.jpg -> 793.11 KB
   → Comprimida: 792.99 KB
JPEG_20250916_112005_7434297881446057555.jpg -> 947.35 KB
   → Comprimida: 947.31 KB
JPEG_20250917_080303_116016971314641476.jpg -> 2316.74 KB
   → Comprimida: 2316.08 KB
JPEG_20250904_104626_5068251037143720812.jpg -> 1685.93 KB
   → Comprimida: 1685.78 KB
JPEG_20250917_102827_299399395532826017.jpg -> 456.71 KB
   → Comprimida: 456.65 K

KeyboardInterrupt: 

In [4]:
import os
import shutil
from PIL import Image


# Carpetas
carpeta = r"C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas"
carpeta_salida = r"C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas_2"
os.makedirs(carpeta_salida, exist_ok=True)
# Límite en KB
limite_kb = 49

# Archivos pendientes
archivos_entrada = {f for f in os.listdir(carpeta) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}
archivos_salida = {f for f in os.listdir(carpeta_salida) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}
pendientes = archivos_entrada - archivos_salida

print(f"Total imágenes en carpeta: {len(archivos_entrada)}")
print(f"Ya procesadas: {len(archivos_salida)}")
print(f"Pendientes: {len(pendientes)}")

def comprimir_imagen(ruta, destino, limite_kb, calidad_inicial=85, paso=5, factor_resize=0.9):
    """
    Comprime una imagen reduciendo calidad y dimensiones hasta que
    esté por debajo del límite en KB.
    """
    try:
        img = Image.open(ruta)

        # Si es PNG → convertir a JPG (mejor compresión)
        if ruta.lower().endswith(".png"):
            img = img.convert("RGB")
            destino = os.path.splitext(destino)[0] + ".jpg"

        calidad = calidad_inicial
        ancho, alto = img.size

        while True:
            # Guardar temporalmente
            img.save(destino, optimize=True, quality=calidad)
            tamano_kb = os.path.getsize(destino) / 1024

            if tamano_kb <= limite_kb:
                return tamano_kb

            # Bajar calidad primero
            if calidad > 20:
                calidad -= paso
            else:
                # Si la calidad ya está muy baja, reducimos dimensiones
                ancho = int(ancho * factor_resize)
                alto = int(alto * factor_resize)
                img = img.resize((ancho, alto), Image.LANCZOS)

            # Si llega a ser muy pequeña y no logra el peso, salimos
            if ancho < 200 or alto < 200:
                return tamano_kb

    except Exception as e:
        print(f"   Error al comprimir {ruta}: {e}")
        return None

# Procesar archivos pendientes
for archivo in pendientes:
    ruta = os.path.join(carpeta, archivo)
    destino = os.path.join(carpeta_salida, archivo)

    if os.path.isfile(ruta):
        tamano_kb = os.path.getsize(ruta) / 1024
        print(f"{archivo} -> {tamano_kb:.2f} KB")

        if tamano_kb <= limite_kb:
            shutil.copy2(ruta, destino)
            print(f"   → Copiada sin cambios ({tamano_kb:.2f} KB)")
        else:
            nuevo_tamano = comprimir_imagen(ruta, destino, limite_kb)
            if nuevo_tamano:
                print(f"   → Comprimida: {nuevo_tamano:.2f} KB")


Total imágenes en carpeta: 523
Ya procesadas: 95
Pendientes: 428
JPEG_20250917_112528_8514050483002133982.jpg -> 2048.00 KB
   Error al comprimir C:\Users\ingju\OneDrive - Cenyt ingenieros\Desktop\Anda\Comprimidas\JPEG_20250917_112528_8514050483002133982.jpg: image file is truncated (73 bytes not processed)
JPEG_20250917_080332_121774368355761287.jpg -> 1219.93 KB
   → Comprimida: 42.01 KB
IMG_20250916_145302_9.jpg -> 165.77 KB
   → Comprimida: 41.65 KB
IMG_20250915_171046_2.jpg -> 242.87 KB
   → Comprimida: 47.96 KB
JPEG_20250904_104701_6614820806790020965.jpg -> 999.90 KB
   → Comprimida: 44.96 KB
IMG_20250916_110018_3.jpg -> 88.27 KB
   → Comprimida: 47.03 KB
IMG_20250917_105546_0.jpg -> 2670.95 KB
   → Comprimida: 46.91 KB
JPEG_20250917_082724_164056103689872441.jpg -> 2419.14 KB
   → Comprimida: 48.89 KB
IMG_20250915_154833_1.jpg -> 419.37 KB
   → Comprimida: 46.90 KB
JPEG_20250916_152359_1150534399233289806.jpg -> 485.85 KB
   → Comprimida: 42.27 KB
JPEG_20250916_094708_723221242

KeyboardInterrupt: 